In [1]:
import pandas as pd
import re
import string
import unicodedata
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
X_train = pd.read_csv('X_train_update.csv')
y_train = pd.read_csv('Y_train_CVw08PX.csv')

df = X_train.copy()
df['prdtypecode'] = y_train['prdtypecode']

In [3]:
stopwords_fr = set([
    "le", "la", "les", "un", "une", "des", "de", "du", "au", "aux", "en",
    "et", "à", "pour", "par", "avec", "sur", "dans", "ce", "ces", "se", "sa",
    "son", "ses", "qui", "que", "quoi", "dont", "où", "comme", "est", "sont",
    "il", "elle", "ils", "elles", "nous", "vous", "ne", "pas", "plus", "moins",
    "ou", "mais", "donc", "or", "ni", "car"
])

In [4]:
def remove_accents(text):
    return ''.join(
        c for c in unicodedata.normalize('NFKD', text)
        if not unicodedata.combining(c)
    )

In [5]:
def nettoyage(row):
    text = row['description']

    if pd.isnull(text) or str(text).strip() == "":
        text = row['designation']
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = remove_accents(text)
    text = re.sub(r'<[^>]+>', ' ', text)  # supprimer HTML
    text = re.sub(r'[^a-zàâçéèêëîïôûùüÿñæœ\s]', ' ', text)  # garder lettres
    text = re.sub(r'\s+', ' ', text).strip()  # espaces propres
    tokens = text.split()
    tokens = [t for t in tokens if t not in stopwords_fr and len(t) > 2]
    return " ".join(tokens)



df['description_propre'] = df.apply(nettoyage, axis=1) #application du nettoyage à la colonne description

print("Aperçu des descriptions nettoyées :")
print(df[['description_propre']].head(10))

Aperçu des descriptions nettoyées :
                                  description_propre
0  olivia personalisiertes notizbuch seiten punkt...
1  journal arts art marche salon art asiatique pa...
2  pilot style touch pen marque speedlink stylet ...
3  peluche donald europe disneyland marionnette d...
4  luc eacute grandeur veut organiser jeu guerre ...
5  afrique contemporaine hiver dossier japon afrique
6            christof bildungsprozessen auf der spur
7  conquerant classique cahier seyes incolorecouv...
8                    puzzle scooby doo poster pieces
9  tente pliante pro pvc soyez particulier votre ...


In [6]:
#Vectorisation

tfidf = TfidfVectorizer(
    max_features=10000,       
    ngram_range=(1, 2),       
    stop_words='english'      
)

X_tfidf = tfidf.fit_transform(df['description_propre'])

print("TF-IDF shape :", X_tfidf.shape)

TF-IDF shape : (84916, 10000)


In [7]:
feature_names = tfidf.get_feature_names_out()
sample_tfidf = X_tfidf[:5].toarray()
df_tfidf_sample = pd.DataFrame(sample_tfidf, columns=feature_names)
df_tfidf_sample = df_tfidf_sample.loc[:, (df_tfidf_sample != 0).any(axis=0)]
print("Exemple fictif de matrice TF-IDF :")
print(df_tfidf_sample.head())

Exemple fictif de matrice TF-IDF :
    adapte  adapte votre   adhesif       ans   arriere       art      arts  \
0  0.00000      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1  0.00000      0.000000  0.000000  0.182861  0.000000  0.500922  0.297919   
2  0.07023      0.114624  0.088852  0.000000  0.070574  0.000000  0.000000   
3  0.00000      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.00000      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

    aussi      bleu     bonus  ...   tactile     tenue     touch      tous  \
0  0.0000  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000   
1  0.0000  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000   
2  0.0633  0.059271  0.110711  ...  0.078763  0.101049  0.198196  0.000000   
3  0.0000  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000   
4  0.0000  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.142402   

        tre     tres      v

In [ ]:
#Extraire les mots les plus fréquents pour chaque prdtypecode
keywords_by_prdtypecode = {}
for code in df['prdtypecode'].unique():
    description_propre = df[df['prdtypecode'] == code]['description_propre'].sum()
    freq = Counter(description_propre).most_common(10)
    keywords_by_prdtypecode[code] = freq

for code, keywords in sorted(keywords_by_prdtypecode.items()):
    print(f"\n🔹 prdtypecode = {code}")
    for word, count in keywords:
        print(f"{word:15} → {count} fois")

KeyError: 'tokens'